In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import datetime as dt
%matplotlib inline
sns.set_context('notebook')

path = '/'.join(os.getcwd().split('/')[:-2])
print(path)

tci = pd.read_csv(path+'/data/model_data/tci_1_0.csv', parse_dates=['Date'], dtype={'PIN': str})
cols = ['parcel','vacant','Date','Survey Category','Survey Grade','SPA_NAME']
tci = tci[cols]
tci.shape 

/Volumes/Dropbox/largetransfer/luc/carter


(113132, 6)

In [2]:
rng = pd.date_range('10/1/2009', periods=23, freq='3MS')

In [5]:
# violations
vs = pd.read_csv(path+'/data/clean_data/violations.csv', parse_dates = [1,4])
# complaints
cp = pd.read_csv(path+'/data/clean_data/complaints.csv', parse_dates = [2], index_col=[0], dtype=str)
cp = cp.sort_values('c_file_date', ascending=True)

In [8]:
for date in rng:

    # get last entry for each ID number
    vs_copy = vs[vs.v_file_date<=date].groupby('VIN').last().reset_index()
    # get the total number of violations for the last year
    vs_copy2 = vs_copy.loc[vs_copy.v_file_date > date-pd.DateOffset(years=1), ['parcel','v_file_date']]\
                    .groupby('parcel').count()
    vs_copy2 = vs_copy2.rename(columns={'v_file_date':'v_total_1yr'})
    # merge total into df
    df = pd.merge(tci, vs_copy2[['v_total_1yr']], \
               how='left', left_on='parcel', right_index=True)
    df.loc[df.v_total_1yr.isnull(), 'v_total_1yr'] = 0
    #  get the total number of violations in last 2 years
    vs_copy2 = vs_copy.loc[vs_copy.v_file_date > date-pd.DateOffset(years=2), ['parcel','v_file_date']]\
                .groupby('parcel').count()
    vs_copy2 = vs_copy2.rename(columns={'v_file_date':'v_total_2yr'})
    # merge into df
    df = pd.merge(df, vs_copy2[['v_total_2yr']], \
               how='left', left_on='parcel', right_index=True)
    df.loc[df.v_total_2yr.isnull(), 'v_total_2yr'] = 0
    
    # get complaints in time period
    cp_copy = cp[(cp.c_file_date<=date) & (cp.c_file_date > dt.datetime(2006,1,1))]
#     &(cp.c_type_of_complaint=='O.V.V.')]\
#                     .groupby('complaint_id').last().reset_index()
    # cp_copy2 = cp_copy[['parcel','c_file_date']].groupby('parcel').last().reset_index()
    # get parcels that have had a closed-board up request during the time period
    df.loc[df.parcel.isin(cp_copy.loc[(cp_copy.c_wf_task_status=='Closed-Board Up Requested'),'parcel']),'boarded'] = 1
    df.loc[df.boarded.isnull(),'boarded'] = 0
    
    #     df = pd.merge(df, cp_copy.loc[(cp_copy.c_wf_task_status=='Closed-Board Up Requested'),\
#                                   ['c_file_date','parcel']]\
#                    .groupby('parcel')\
#                    .last()\
#                         ,how='left', left_on='parcel', right_index=True)
#     df = df.rename(columns = {'c_file_date':'days_since_ovv_board_up'})
    
#     df.loc[df.days_since_ovv_board_up.isnull(),'days_since_ovv_board_up'] = dt.datetime(2006,1,1)
#     df['days_since_ovv_board_up'] = tci.apply(lambda x: (x.Date-x.days_since_ovv_board_up).days, axis=1)

#     df = pd.merge(df, cp_copy.loc[(cp_copy.c_file_date > dt.datetime(2006,1,1))\
#                                     &(cp.c_wf_task_status=='Closed-Board Up Requested'),['c_file_date','parcel']]\
#                    .groupby('parcel')\
#                    .agg(lambda x: len(x)\
#                         /((dates[x.parcel.iloc[0]] - dt.datetime(2006,1,1)).days/365.0))\
#                         ,how='left', left_on='parcel', right_index=True)
#     tci = tci.rename(columns = {'c_file_date':'c_ovv_board_num'})
#     tci.loc[tci.c_ovv_board_num.isnull(),'c_ovv_board_num'] = 0
    cp_copy = cp_copy.groupby('complaint_id').last().reset_index()
    # days since most recent OVV complaint
    df = pd.merge(df, cp_copy.loc[(cp_copy.c_type_of_complaint=='O.V.V.')\
                                  ,['c_file_date','parcel']]\
                   .groupby('parcel')\
                   .last(), how='left', left_on='parcel', right_index=True)
    df = df.rename(columns = {'c_file_date':'days_since_ovv'})
    df.loc[df.days_since_ovv.isnull(),'days_since_ovv'] = dt.datetime(2006,1,1)
    df['days_since_ovv'] = df.apply(lambda x: (date-x.days_since_ovv).days, axis=1)
    
    # average number of OVV complaints per year since 2006
    df = pd.merge(df, cp_copy.loc[(cp_copy.c_file_date > dt.datetime(2006,1,1)) & (cp_copy.c_type_of_complaint=='O.V.V.'),\
                                  ['c_file_date','parcel']]\
                   .groupby('parcel')\
                   .agg(lambda x: len(x)\
                        /((date - dt.datetime(2006,1,1)).days/365.0))\
                        ,how='left', left_on='parcel', right_index=True)
    df = df.rename(columns = {'c_file_date':'c_ovv_num'})
    df.loc[df.c_ovv_num.isnull(),'c_ovv_num'] = 0
#     break
    df.to_csv(path+'/data/model_data/tci_2_2_'+str(date.year)+'_'+str(date.month)+'.csv', index=False)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113132 entries, 0 to 113131
Data columns (total 11 columns):
parcel             113132 non-null object
vacant             113132 non-null int64
Date               113132 non-null datetime64[ns]
Survey Category    113132 non-null object
Survey Grade       113132 non-null object
SPA_NAME           113132 non-null object
v_total_1yr        113132 non-null float64
v_total_2yr        113132 non-null float64
boarded            113132 non-null float64
days_since_ovv     113132 non-null int64
c_ovv_num          113132 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 10.4+ MB
